
# **Initiate and Configure Spark**

---


In [ ]:
########## ONLY in Colab ##########
!pip3 install pyspark
########## ONLY in Colab ##########

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 199 kB 65.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=c8b4447df5572e64583df6e18bd3042ae0355798c1684a92af3d76320d82428b
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
########## ONLY in Ubuntu Machine ##########
# Load Spark engine
!pip3 install -q findspark
import findspark
findspark.init()
########## ONLY in Ubuntu Machine ##########

In [ ]:
# linking with SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('Your Group ID').getOrCreate()

# Note: If you want to work with RDD, you should use: "from pyspark import SparkContext, SparkConf"

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
  .builder \
  .appName("Python Spark SQL basic example") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

In [ ]:
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
import pandas as pd
from pyspark.sql.functions import lit, desc, col, size, array_contains\
, isnan, udf, hour, array_min, array_max, countDistinct
from pyspark.sql.types import *
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql.functions import isnan, when, count, col, translate
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql import functions as F
from pyspark import SparkConf, SparkContext, SQLContext
conf = SparkConf().setAppName("test").setMaster("local")
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)


# **Load Data**

---


In [ ]:
########## ONLY in Colab ##########
# mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')
########## ONLY in Colab ##########

Mounted at /content/drive


In [ ]:
# Load Data from Colab:

########## ONLY in Colab ##########
IDS_df = spark.read.option("header", "true").csv("/content/drive/MyDrive/Colab Notebooks/IDS2018/*.csv")
########## ONLY in Colab ##########

In [ ]:
# Load Data from Ubuntu VMware:

########## ONLY in Ubuntu Machine ##########
IDS_df = spark.read.option("header", "true").csv("/content/drive/MyDrive/Colab Notebooks/IDS2018/*.csv",inferSchema=True)
########## ONLY in Ubuntu Machine ##########

In [ ]:
# Print DF to make sure it is working
IDS_df.show()

+--------+--------+-------------------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+---------------+-------------+----------------+----------------+------------+------------+------------+----------------+----------------+-----------+-----------+-----------+----------------+----------------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+------------+-----------+-----------+--------------+--------------+----------------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+--------------+----------------+----------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+----------------+----------------+----------------+-------------

In [ ]:
# The total number of attacks per label
IDS_df.select('Label').groupBy('Label').count().orderBy('count', ascending=False).show()

+--------------------+-------+
|               Label|  count|
+--------------------+-------+
|              Benign|5329908|
|    DDOS attack-HOIC| 686012|
|    DoS attacks-Hulk| 461912|
|                 Bot| 286191|
|      FTP-BruteForce| 193360|
|      SSH-Bruteforce| 187589|
|DoS attacks-SlowH...| 139890|
|DoS attacks-Golde...|  41508|
|DoS attacks-Slowl...|  10990|
|DDOS attack-LOIC-UDP|   1730|
|    Brute Force -Web|    611|
|    Brute Force -XSS|    230|
|       SQL Injection|     87|
+--------------------+-------+




# **Task 1: Spark SQL [30 marks]**

---



In [ ]:
# Student 1 name and ID
# Query 1 [Briefly explain]: The application of the group by function has been done in this particular case scenario and the application of this particular function has been done on the particular column “protocol”. The different protocols that are being visualised here are “0”, “6” and “17”. 
IDS_df.groupBy("Protocol").count().show()

+--------+-------+
|Protocol|  count|
+--------+-------+
|       6|5906072|
|      17|1357656|
|       0|  76290|
+--------+-------+



In [ ]:
# Student 1 name and ID
# Query 2 [Briefly explain]: The visualization of the three particular columns has been done with the help of this particular query application with the anomaly direction. The columns that are being visualised here are “Dst Port”, “label” and  “Flow Duration”. 
IDS_df.select("Dst Port", "Label", "Flow Duration").show()

+--------+------+-------------+
|Dst Port| Label|Flow Duration|
+--------+------+-------------+
|       0|Benign|    112641719|
|       0|Benign|    112641466|
|       0|Benign|    112638623|
|      22|Benign|      6453966|
|      22|Benign|      8804066|
|      22|Benign|      6989341|
|       0|Benign|    112640480|
|       0|Benign|    112641244|
|      80|Benign|       476513|
|      80|Benign|       475048|
|      80|Benign|       474926|
|      80|Benign|       477471|
|      80|Benign|       512758|
|      80|Benign|       476711|
|      80|Benign|       476616|
|      80|Benign|       477161|
|      80|Benign|       474670|
|      80|Benign|       476608|
|      80|Benign|       479249|
|      80|Benign|       475967|
+--------+------+-------------+
only showing top 20 rows



In [ ]:
# Student 2 name and ID
# Query 1 [Briefly explain]: In this particular case scenario the visualisation of the particular query has been done. In the query, it has been mentioned that if the label shows the “benign” then it needs to be  identified as “0”. On the other hand other than that other labels are needs identified as “1”. 
IDS_df.select("Label",when(IDS_df.Label != 'Benign', 1).otherwise(0)).show(150)


+--------------+--------------------------------------------------+
|         Label|CASE WHEN (NOT (Label = Benign)) THEN 1 ELSE 0 END|
+--------------+--------------------------------------------------+
|        Benign|                                                 0|
|        Benign|                                                 0|
|        Benign|                                                 0|
|        Benign|                                                 0|
|        Benign|                                                 0|
|        Benign|                                                 0|
|        Benign|                                                 0|
|        Benign|                                                 0|
|        Benign|                                                 0|
|        Benign|                                                 0|
|        Benign|                                                 0|
|        Benign|                                

In [ ]:
# Student 2 name and ID
# Query 2 [Briefly explain]: The visualization of the particular that that has been present in the data set that has been visualised with the help of the above-mentioned figure. The particular type of attack that has been filtered here are the “Brute force- XSS” and “SQL Injection”. All the fields that are associated with this particular label that has been visualised with the help of this particular query.
IDS_df[IDS_df.Label.isin("Brute Force -XSS", "SQL Injection")].show(5)

+--------+--------+-------------------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+---------------+----------------+----------------+----------------+------------+------------+-----------+----------------+----------------+-----------+-----------+-----------+----------------+----------------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+----------------+------------+-----------+-----------+--------------+--------------+----------------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+--------------+----------------+----------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+----------------+----------------+----------------+--------

In [ ]:
# Student 3 name and ID
# Query 1 [Briefly explain]: In the IDS data frame all the “benign” labels that are present in the data set that has been filtered out with the help of this particular quarry application. At the same time, the visualization of the query has been done in the above figure. All the features that are associated with the “benign” labels are being filtered out here with the help of this particular quary. 
IDS_df.createOrReplaceTempView("IDS")
sqlDF = spark.sql("SELECT * FROM IDS where Label = 'Benign'")
sqlDF.show()


+--------+--------+-------------------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+---------------+-------------+----------------+----------------+------------+------------+------------+----------------+----------------+-----------+-----------+-----------+----------------+----------------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+------------+-----------+-----------+--------------+--------------+----------------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+--------------+----------------+----------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+----------------+----------------+----------------+-------------

In [ ]:
# Student 3 name and ID
# Query 2 [Briefly explain]: This particular quarry has been done where the filtration of the “protocol” features has been done where at the same time the condition that has been applied in the query is that the protocol labels that are greater than “0” are being filtered here. So all the fields that are present in the IDS data frame that is associated with this particular condition that has been filtered out and then all the features are visualised with the help of this query. 
sqlDF1 = spark.sql("SELECT * FROM IDS where Protocol > '0'")
sqlDF1.show()

+--------+--------+-------------------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+---------------+-------------+----------------+----------------+------------+------------+-----------+----------------+----------------+-----------+-----------+-----------+----------------+----------------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+------------+-----------+-----------+--------------+--------------+----------------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+--------------+----------------+----------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+----------------+----------------+----------------+--------------

In [ ]:
# Student 4 name and ID
# Query 1 [Briefly explain]: The particular time stamp that is present in the data set has been visualised with the help of this particular quarry application. The particular timestamp that has been given here is dated “14/02/2018” and the time steam is “08:40:13”. At this given range all the data that are captured at that time and present in the data frame have been visualised here. 

sqlDF2 = spark.sql("""SELECT * FROM IDS where  Timestamp  == '14/02/2018 08:40:13'""")
sqlDF2.show()

+--------+--------+-------------------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+----------------+----------------+----------------+------------------+------------+------------+------------+------------------+------------------+------------+-----------+------------+----------------+------------------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+----------------+----------------+-----------+-----------+--------------+--------------+----------------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+--------------+----------------+----------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+----------------+----------------+---------

In [ ]:
# Student 4 name and ID
# Query 2 [Briefly explain]: The maximum value of the “protocol” column that is present in the IDS data frame has been visualised with the help of this particular query application. The minimum number of protocols that have been visualised here is 17.  

sqlDF3 = spark.sql("""SELECT MAX(Protocol) FROM IDS""")
sqlDF3.show()

+-------------+
|max(Protocol)|
+-------------+
|           17|
+-------------+




# **Task 2 - Part1: PySpark [45 marks]**

---


In [ ]:
# Student 1 name and ID
# Analytical method 1:
IDS_df.printSchema()

root
 |-- Dst Port: integer (nullable = true)
 |-- Protocol: integer (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Flow Duration: long (nullable = true)
 |-- Tot Fwd Pkts: integer (nullable = true)
 |-- Tot Bwd Pkts: integer (nullable = true)
 |-- TotLen Fwd Pkts: integer (nullable = true)
 |-- TotLen Bwd Pkts: double (nullable = true)
 |-- Fwd Pkt Len Max: integer (nullable = true)
 |-- Fwd Pkt Len Min: integer (nullable = true)
 |-- Fwd Pkt Len Mean: double (nullable = true)
 |-- Fwd Pkt Len Std: double (nullable = true)
 |-- Bwd Pkt Len Max: integer (nullable = true)
 |-- Bwd Pkt Len Min: integer (nullable = true)
 |-- Bwd Pkt Len Mean: double (nullable = true)
 |-- Bwd Pkt Len Std: double (nullable = true)
 |-- Flow Byts/s: double (nullable = true)
 |-- Flow Pkts/s: double (nullable = true)
 |-- Flow IAT Mean: double (nullable = true)
 |-- Flow IAT Std: double (nullable = true)
 |-- Flow IAT Max: double (nullable = true)
 |-- Flow IAT Min: double (nullable = true)

In [ ]:
# Student 1 name and ID
# Analytical method 2:

IDS_df.describe().toPandas()

,summary,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,count,7340018,7340018,7340018,7340018,7340018,7340018,7340018,7340018,7340018,...,7340018,7340018,7340018,7340018,7340018,7340018,7340018,7340018,7340018,7340018
1,mean,11231.730779679287,7.972267097982593,None,9795900.24853345,32.943601500704766,5.9616829277530385,1293.6394789495066,4608.409493001243,238.01234710868556,...,20.421886158862282,61414.38704432067,28904.194980478653,102495.23225392636,43885.050490339396,4913482.526620346,405713.6812994857,5632559.51359697,4392848.859654976,None
2,stddev,20483.768566117982,4.3434158568247305,None,7.337834016610662E8,1854.0592777044114,171.3252494015615,60613.5243447207,244707.37840358494,344.25881122072633,...,8.509632803828069,981352.8993974862,550744.2749961028,1311250.3100639815,851165.225199727,3.9123126257032514E8,2.515981318870479E8,9.298528373349414E8,9.319939330716385E7,None
3,min,0,0,02/03/2018 01:00:00,-919011000000,1,0,0,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,max,65534,17,23/02/2018 12:59:59,120000000,309629,123118,9908128,1.56360426E8,64440,...,56,1.13170569E8,7.49E7,1.13170569E8,1.13170569E8,3.95571421052631E11,2.62247866338599E11,9.79781E11,2.39934E11,SSH-Bruteforce


In [ ]:
# Student 1 name and ID
# Analytical method 3:
from pyspark.sql.functions import *
IDS_df.stat.corr("Tot Fwd Pkts", "Tot Bwd Pkts")

0.027713649734639596

In [ ]:
# Student 2 name and ID
# Analytical method 1:

from pyspark.ml.feature import StringIndexer, OneHotEncoder
indexer = StringIndexer(inputCol='Label', outputCol='Label_numeric').fit(IDS_df)
IDS_df_1 = indexer.transform(IDS_df)

In [ ]:
IDS_df_1.show()

+--------+--------+-------------------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+---------------+-------------+----------------+----------------+------------+------------+------------+----------------+----------------+-----------+-----------+-----------+----------------+----------------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+-------------+------------+-----------+-----------+--------------+--------------+----------------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+--------------+----------------+----------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+----------------+----------------+----------------+-------------

In [ ]:
# Student 2 name and ID
# Analytical method 2:

IDS_df.stat.corr("Flow Duration", "Protocol")

-0.005496709809106347

In [ ]:
# Student 2 name and ID
# Analytical method 3:

from pyspark.ml.feature import VectorAssembler

IDS_df_1 = IDS_df_1.drop("Timestamp", "Label")
IDS_df_1.printSchema()

root
 |-- Dst Port: integer (nullable = true)
 |-- Protocol: integer (nullable = true)
 |-- Flow Duration: long (nullable = true)
 |-- Tot Fwd Pkts: integer (nullable = true)
 |-- Tot Bwd Pkts: integer (nullable = true)
 |-- TotLen Fwd Pkts: integer (nullable = true)
 |-- TotLen Bwd Pkts: double (nullable = true)
 |-- Fwd Pkt Len Max: integer (nullable = true)
 |-- Fwd Pkt Len Min: integer (nullable = true)
 |-- Fwd Pkt Len Mean: double (nullable = true)
 |-- Fwd Pkt Len Std: double (nullable = true)
 |-- Bwd Pkt Len Max: integer (nullable = true)
 |-- Bwd Pkt Len Min: integer (nullable = true)
 |-- Bwd Pkt Len Mean: double (nullable = true)
 |-- Bwd Pkt Len Std: double (nullable = true)
 |-- Flow Byts/s: double (nullable = true)
 |-- Flow Pkts/s: double (nullable = true)
 |-- Flow IAT Mean: double (nullable = true)
 |-- Flow IAT Std: double (nullable = true)
 |-- Flow IAT Max: double (nullable = true)
 |-- Flow IAT Min: double (nullable = true)
 |-- Fwd IAT Tot: double (nullable = tru

In [ ]:
IDS_df_1.stat.cov("Fwd Seg Size Min", "Bwd Blk Rate Avg")

0.0

In [ ]:
# Student 3 name and ID
# Analytical method 1:

IDS_df_1.stat.crosstab("Dst Port", "Bwd Pkts/b Avg").show()

+-----------------------+---+
|Dst Port_Bwd Pkts/b Avg|  0|
+-----------------------+---+
|                  53767| 53|
|                  32571|  4|
|                  42890| 30|
|                  46805|  2|
|                  37159|  5|
|                  29176|  4|
|                  14221|  4|
|                   2163|  3|
|                  37041|  4|
|                  62232| 24|
|                   8607|  2|
|                  60426| 58|
|                  25695|  4|
|                  31843|  2|
|                  20873|  1|
|                  23162|  5|
|                  39241|  7|
|                  64790| 31|
|                  42900| 46|
|                  46827|  4|
+-----------------------+---+
only showing top 20 rows



In [ ]:
# Student 3 name and ID
# Analytical method 2:

IDS_df_1.stat.corr("Label_numeric", "Dst Port")

-0.23319771049936172

In [ ]:
# Student 3 name and ID
# Analytical method 3:

IDS_df_1.select(stddev('Flow Duration')).show()

+--------------------------+
|stddev_samp(Flow Duration)|
+--------------------------+
|       7.337834016610662E8|
+--------------------------+



In [ ]:
# Student 4 name and ID
# Analytical method 1:

IDS_df_1.stat.freqItems(["Pkt Len Min", "ACK Flag Cnt"], 0.5).show()

+---------------------+----------------------+
|Pkt Len Min_freqItems|ACK Flag Cnt_freqItems|
+---------------------+----------------------+
|              [32, 0]|                [1, 0]|
+---------------------+----------------------+



In [ ]:
# Student 4 name and ID
# Analytical method 2:

sample1 = IDS_df_1.sample(True, 0.5)
sample2 = IDS_df_1.sample(True, 0.5)

sample_1_mean = sample1.select(avg('Protocol')).collect()[0][0]
sample_2_mean = sample2.select(avg('Dst Port')).collect()[0][0]

diff_in_means = sample_1_mean - sample_2_mean

sample_1_std = sample1.select(stddev('Protocol')).collect()[0][0]
sample_2_std = sample2.select(stddev('Dst Port')).collect()[0][0]

standard_error = (sample_1_std + sample_2_std) / 2

t_statistic = diff_in_means / standard_error

from scipy.stats import t
p_value = t.sf(t_statistic, IDS_df_1.count()-1)*2

significance_level = 0.05
if p_value < significance_level:
   print('Reject the null hypothesis. There is a significant difference between the two samples.')
else:
   print('Fail to reject the null hypothesis. There is no significant difference between the two samples.')

Fail to reject the null hypothesis. There is no significant difference between the two samples.


In [ ]:
# Student 4 name and ID
# Analytical method 3:
IDS_df.stat.corr("Idle Min", "Active Mean")


0.009410777030244883


# **Task 2 - Part2: PySpark [15 marks]**

---


In [ ]:
# Student 1 name and ID
# Machine Learning Technique:
# What to achieve: Random Forest Classifier

assembler = VectorAssembler(inputCols=['Dst Port',
 'Protocol',
 'Flow Duration'], outputCol="features")
data = assembler.transform(IDS_df_1)


In [ ]:
data.select(["features","Label_numeric"]).show(truncate=False)

+----------------------+-------------+
|features              |Label_numeric|
+----------------------+-------------+
|[0.0,0.0,1.12641719E8]|0.0          |
|[0.0,0.0,1.12641466E8]|0.0          |
|[0.0,0.0,1.12638623E8]|0.0          |
|[22.0,6.0,6453966.0]  |0.0          |
|[22.0,6.0,8804066.0]  |0.0          |
|[22.0,6.0,6989341.0]  |0.0          |
|[0.0,0.0,1.1264048E8] |0.0          |
|[0.0,0.0,1.12641244E8]|0.0          |
|[80.0,6.0,476513.0]   |0.0          |
|[80.0,6.0,475048.0]   |0.0          |
|[80.0,6.0,474926.0]   |0.0          |
|[80.0,6.0,477471.0]   |0.0          |
|[80.0,6.0,512758.0]   |0.0          |
|[80.0,6.0,476711.0]   |0.0          |
|[80.0,6.0,476616.0]   |0.0          |
|[80.0,6.0,477161.0]   |0.0          |
|[80.0,6.0,474670.0]   |0.0          |
|[80.0,6.0,476608.0]   |0.0          |
|[80.0,6.0,479249.0]   |0.0          |
|[80.0,6.0,475967.0]   |0.0          |
+----------------------+-------------+
only showing top 20 rows



In [ ]:
RF=data.select(["features", "Label_numeric"])

In [ ]:
trainingdf, testingdf=RF.randomSplit([0.7,0.3])

In [ ]:
print("Training data", trainingdf.count())
print("Testing data", testingdf.count())

Training data 5136052
Testing data 2203966


In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
rfclass=RandomForestClassifier(labelCol="Label_numeric", numTrees=5).fit(trainingdf)
rfpred=rfclass.transform(testingdf)
rfpred.show()

+--------------------+-------------+--------------------+--------------------+----------+
|            features|Label_numeric|       rawPrediction|         probability|prediction|
+--------------------+-------------+--------------------+--------------------+----------+
|[0.0,0.0,-2.7385E11]|          0.0|[2.00234053757703...|[0.40046810751540...|       0.0|
|[0.0,0.0,-1.1873E10]|          0.0|[2.00234053757703...|[0.40046810751540...|       0.0|
|      [0.0,0.0,97.0]|          0.0|[3.75019611309380...|[0.75003922261876...|       0.0|
|     [0.0,0.0,105.0]|          0.0|[3.75019611309380...|[0.75003922261876...|       0.0|
|     [0.0,0.0,107.0]|          0.0|[3.75019611309380...|[0.75003922261876...|       0.0|
|     [0.0,0.0,132.0]|          0.0|[3.75019611309380...|[0.75003922261876...|       0.0|
|     [0.0,0.0,135.0]|          0.0|[3.75019611309380...|[0.75003922261876...|       0.0|
|     [0.0,0.0,171.0]|          0.0|[3.75019611309380...|[0.75003922261876...|       0.0|
|     [0.0

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
rfaccuracy= (MulticlassClassificationEvaluator(labelCol="Label_numeric").evaluate(rfpred))*100

In [ ]:
print("The accuracy of the Random Forest Classifier model is:", rfaccuracy)

The accuracy of the Random Forest Classifier model is: 90.12136375829391


In [ ]:
from sklearn.metrics import confusion_matrix
y_pre=rfpred.select("prediction").collect()
y_ori=rfpred.select("Label_numeric").collect()

cm = confusion_matrix(y_ori, y_pre)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[1581464   11504    7171      27       3     316       0       0       0
        0       0       0       0]
 [  12461  176348   16617       0       0       0       0       0       0
        0       0       0       0]
 [  29645   66764   42517       0       0       0       0       0       0
        0       0       0       0]
 [   1382       0       0   84284       0       0       0       0       0
        0       0       0       0]
 [      0       0       0       0   51315       0    6484       0       0
        0       0       0       0]
 [      3       0       0       0       7   56105       1       0       0
        0       0       0       0]
 [      0       0       0       0   23502       0   18372       0       0
        0       0       0       0]
 [  12250       1     173       0       0       0       0       0       0
        0       0       0       0]
 [   3296       9       0       0       0       0       0       0       0
        0       0       0       0]
 

In [ ]:
# Student 2 name and ID
# Machine Learning Technique:
# What to achieve: Logistic Regression
from pyspark.ml.classification import LogisticRegression


In [ ]:
logistic = LogisticRegression(labelCol="Label_numeric").fit(trainingdf)

In [ ]:
prediction = logistic.transform(testingdf)
prediction.groupBy("Label_numeric", "prediction").count().show()

+-------------+----------+-------+
|Label_numeric|prediction|  count|
+-------------+----------+-------+
|          0.0|       1.0|  22852|
|          0.0|       0.0|1577632|
|          0.0|       2.0|      1|
|          4.0|       0.0|  57799|
|          5.0|       0.0|  56116|
|          8.0|       0.0|   3305|
|          7.0|       0.0|  12424|
|          2.0|       0.0| 138926|
|          6.0|       0.0|  41874|
|          9.0|       0.0|    530|
|          1.0|       0.0| 205426|
|         10.0|       0.0|    191|
|         12.0|       0.0|     33|
|         11.0|       0.0|     66|
|         10.0|       1.0|      5|
|          3.0|       1.0|     70|
|          3.0|       0.0|  85596|
+-------------+----------+-------+



In [ ]:
LRaccuracy= (MulticlassClassificationEvaluator(labelCol="Label_numeric").evaluate(prediction))*100

In [ ]:
print("The accuracy of the Logistic Regression model is:", LRaccuracy)

The accuracy of the Random Forest Classifier model is: 60.64082129893443


In [ ]:
# Student 3 name and ID
# Machine Learning Technique:
# What to achieve: Decision Tree Classifier

from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
datatest = DecisionTreeClassifier(labelCol="Label_numeric", featuresCol="features",impurity='entropy', maxDepth=4,seed=1234).fit(trainingdf)
predictions = datatest.transform(testingdf)

In [ ]:
evaluator = MulticlassClassificationEvaluator(\
labelCol="Label_numeric", predictionCol="prediction",\
metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("The accuracy of Decision Tree Classifier model is:" , accuracy)
print(datatest.toDebugString)

The accuracy of Decision Tree Classifier model is: 0.8312387702090841
DecisionTreeClassificationModel: uid=DecisionTreeClassifier_330451519ab3, depth=4, numNodes=23, numClasses=13, numFeatures=3
  If (feature 0 <= 22.5)
   If (feature 0 <= 21.5)
    If (feature 2 <= 42.5)
     If (feature 2 <= 2.5)
      Predict: 4.0
     Else (feature 2 > 2.5)
      Predict: 6.0
    Else (feature 2 > 42.5)
     Predict: 0.0
   Else (feature 0 > 21.5)
    If (feature 2 <= 595118.0)
     Predict: 5.0
    Else (feature 2 > 595118.0)
     Predict: 0.0
  Else (feature 0 > 22.5)
   If (feature 0 <= 101.5)
    If (feature 1 <= 11.5)
     If (feature 2 <= 256572.0)
      Predict: 1.0
     Else (feature 2 > 256572.0)
      Predict: 0.0
    Else (feature 1 > 11.5)
     If (feature 0 <= 73.5)
      Predict: 0.0
     Else (feature 0 > 73.5)
      Predict: 9.0
   Else (feature 0 > 101.5)
    If (feature 2 <= 80765.0)
     If (feature 0 <= 8081.5)
      Predict: 3.0
     Else (feature 0 > 8081.5)
      Predict: 0.0

In [ ]:
# Student 4 name and ID
# Machine Learning Technique:
# What to achieve: K-Means Clustering

from pyspark.ml.feature import VectorAssembler
cols = ["Dst Port", "Protocol", "Flow Duration", "Label_numeric"]

assembler=VectorAssembler(inputCols=cols, outputCol="features")
dataset = assembler.transform(IDS_df_1)
dataset.printSchema()
dataset.show(10) 


root
 |-- Dst Port: integer (nullable = true)
 |-- Protocol: integer (nullable = true)
 |-- Flow Duration: long (nullable = true)
 |-- Tot Fwd Pkts: integer (nullable = true)
 |-- Tot Bwd Pkts: integer (nullable = true)
 |-- TotLen Fwd Pkts: integer (nullable = true)
 |-- TotLen Bwd Pkts: double (nullable = true)
 |-- Fwd Pkt Len Max: integer (nullable = true)
 |-- Fwd Pkt Len Min: integer (nullable = true)
 |-- Fwd Pkt Len Mean: double (nullable = true)
 |-- Fwd Pkt Len Std: double (nullable = true)
 |-- Bwd Pkt Len Max: integer (nullable = true)
 |-- Bwd Pkt Len Min: integer (nullable = true)
 |-- Bwd Pkt Len Mean: double (nullable = true)
 |-- Bwd Pkt Len Std: double (nullable = true)
 |-- Flow Byts/s: double (nullable = true)
 |-- Flow Pkts/s: double (nullable = true)
 |-- Flow IAT Mean: double (nullable = true)
 |-- Flow IAT Std: double (nullable = true)
 |-- Flow IAT Max: double (nullable = true)
 |-- Flow IAT Min: double (nullable = true)
 |-- Fwd IAT Tot: double (nullable = tru

In [ ]:
trainingdata, testdata = dataset.randomSplit([0.8, 0.2], seed = 6253)

print("Training dataset: " + str(trainingdata.count()))

print("Testing dataset: " + str(testdata.count()))

Training dataset: 5872433
Testing dataset: 1467585


In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
cluster_count = [10,12,14,16,18]
clustering_results = []
for i in cluster_count:
  kmeans = KMeans().setK(i).setSeed(1).setFeaturesCol("features").setPredictionCol("prediction")
  kmeansModel = kmeans.fit(trainingdata)
  predictdataset = kmeansModel.transform(testdata)
  predictdataset.show(10)
  evaluator = ClusteringEvaluator()  
  silhouette = evaluator.evaluate(predictdataset)
  clustering_results.append(silhouette)
  print("Silhouette with squared euclidean distance = " + str(silhouette))

+--------+--------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+-----------+-------------------+-------------------+-------------------+------------+------------+-----------+-------------------+-------------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+----------------+----------------+-----------+-----------+------------+-----------+-----------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+------------+----------------+----------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+----------------+----------------+----------------+-----------------+----------------

In [ ]:
predictdataset = kmeansModel.transform(testdata)
predictdataset.show(10)
evaluator = ClusteringEvaluator()  
silhouette = evaluator.evaluate(predictdataset)
print("Silhouette with squared euclidean distance = " + str(silhouette))

+--------+--------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+-----------+-------------------+-------------------+-------------------+------------+------------+-----------+-------------------+-------------------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+----------------+----------------+-----------+-----------+------------+-----------+-----------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+------------+----------------+----------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+----------------+----------------+----------------+-----------------+----------------

In [ ]:
predictdataset.groupBy("prediction").count().show()

+----------+-------+
|prediction|  count|
+----------+-------+
|        12|  20471|
|        13|   3379|
|        16|   9324|
|         3|      1|
|        15|  20522|
|         9|  33421|
|        17|   8820|
|         7|  17294|
|        11|  34296|
|        14|  15235|
|         2|      2|
|         0|1304819|
|         8|      1|
+----------+-------+



# **Convert ipynb to HTML for Turnitin submission [10 marks]**

---



In [ ]:
# install nbconvert
!pip3 install nbconvert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# convert ipynb to html
# file name: "Your_Group_ID_CN7031.ipynb
!jupyter nbconvert --to html Your_Group_ID_CN7031.ipynb

[NbConvertApp] WARNING | pattern 'Your_Group_ID_CN7031.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--ex